# 复旦LLM入门文档第二版

# 导读大纲

**认知**
1. 语言模型（Language Model，LM）的目标就是对自然语言的概率分布建模
2. 大预言模型缩放法则， 参数量、数据量、计算量指数增加的过程，会使得模型性能线性增加
3. 基础模型是指仅经过预训练的模型；对话模型是指在预训练模型基
础上经过有监督微调和强化学习训练的模型，具备对话和完成任务的能力；推理模型是指专注于
逻辑推理增强的大语言模型。
4.

**统计学**
1. 最大似然估计
2. N-gram算法
3. 数据平滑、零概率问题


**神经网络**
1. 2000年提出前馈神经网络
2. ELMo
3. Bert、 GPT
4. 2020年发布了GPT3 （Generative Pre-trained Transformer 3）
5. PaLM、LaMDA


**LLM**
1. 根据OpenAI公开的文档，大语言模型训练过程，包含以下四个主要过程  
  + 预训练(Pretraining)：提升分布式训练的效率应当是该阶段最大的难点
  + 有监督微调(Supervised Fine Tuning，SFT)：难点是如何构造少量并且高质量的监督数据
  + 奖励建模(Reward Modeling)：难点是限定奖励模型应用的泛化边界
  + 强化学习(Reinforcement Learning，RL)：这里相对较复杂，需要仔细研究一下，强化学习在此生效的机制是什么？  
2. 大模型技术版图


| 层级 | 核心目标 | 主流技术 / 方法 | 交叉 & 可选技术 | 常用框架 / 工具 |
| --- | --- | --- | --- | --- |
| **0. 数据 & Tokenizer** | 高质语料收集、去噪、切词 | - Common Crawl / Book / 社媒清洗  <br>- 重采样 (CCMix)  <br>- 语义去重 (MinHash) | - 自动标注 (self-instruct)  <br>- 多语语料对齐 | Pile · OpenWebText · tiktoken · SentencePiece |
| **1. 预训练 (Foundation)** | 学通用语言 / 多模态表征 | - 自回归 / Mask LLM  <br>- MoE 扩容 (GShard / Switch-MoE)  <br>- 多模态对齐 (ALIGN, BLIP-2) | - 低精度优化 (8-bit Adam, FP8)  <br>- 节点并行 (TP / PP) | Megatron-LM · DeepSpeed-Megatron · PaLM-RLHF · Fuyu |
| **2. 对齐训练 (Alignment)** | 让模型“听话 + 讨喜” | - **SFT** (全参 / LoRA / QLoRA)  <br>- **RM** (Pairwise Ranking)  <br>- **RLHF** (PPO-KL) | - **DPO / IPO**  <br>- **RLAIF** (AI-feedback)  <br>- 审校微调 (Refusal / Detox) | HF Trainer · trlX · DeepSpeed-Chat |
| **3. 压缩 & 部署准备** | 降显存、提吞吐 | - 量化 (8 / 4 / 2-bit, GPTQ, AWQ)  <br>- 蒸馏 (TinyLlama, DistilGPT-2)  <br>- FlashAttention-2 / xformers | - 动态 KV Cache (PagedAttention)  <br>- Parameter-Efficient MoE 拆分 | bitsandbytes · auto-gptq · FT-Transformer |
| **4. 推理 & 编排 (Inference / Orchestration)** | 实时问答、工具调用、完成复杂任务 | - **RAG** (向量检索 + Prompt 拼接)  <br>- **Agent / Tool Calling** (ReAct, Auto-GPT)  <br>- 批调度 / vLLM  <br>- Guardrails / Policy-RM 复审 | - 长上下文 (Rope-Scaling, Flash-Infer)  <br>- Memory / Retriever-Augmented Agent | vLLM · TGI · LangChain · Llama-Index · OpenAI Functions |


# 基础知识

## Transformers
1. 位置编码
第一，正余弦函数的范围是 [−1, +1]，导出的位置编码与原词嵌入相加不会使得结果偏离过远而破坏原有单词的语义信息；第二，依据三角函数的基本性质，可以得知第 pos + k 个位置编码是第 pos 个位置编码的线性组合，这就意味着位置编码中蕴含着单词之间的距离信息。  
2.

In [8]:
import torch
import torch.nn as nn
import math

In [22]:
# 三角函数位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, d_model):
        super().__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        pe = torch.zeros(self.max_seq_len, self.d_model)
        for pos in range(self.max_seq_len):
            for i in range(0, self.d_model, 2):
                pe[pos, i] = math.sin(pos / pow(1000, i / self.d_model))
                pe[pos, i + 1] = math.cos(pos / pow(1000, i / self.d_model))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe) # 随参数保存，同时不参与梯度计算

    def forward(self, x):
        x = x * math.sqrt(self.d_model)

        # 叠加位置编码
        x = x + self.pe[:, x.size(1), :] # 如果需要使用显卡，这里需要to device
        return x
pe = PositionalEncoding(8, 24)
print(pe.pe.shape)
print(pe.pe[:, 2, :])

torch.Size([1, 8, 24])
tensor([[ 0.9093, -0.4161,  0.9021,  0.4315,  0.5911,  0.8066,  0.3482,  0.9374,
          0.1987,  0.9801,  0.1122,  0.9937,  0.0632,  0.9980,  0.0356,  0.9994,
          0.0200,  0.9998,  0.0112,  0.9999,  0.0063,  1.0000,  0.0036,  1.0000]])


In [23]:
# 多头自注意力机制

In [25]:
# 前馈神经网络

In [24]:
# 残差网络
# 其实这里需要解释一下，为什么残差链接呢能解决提督消失的问题？


In [32]:
# 层正则化
# 假如维度是(batch_size, seq_len, hidden_dim)，每个样本、每个token都进行单独的处理，但是embedding维度是进行均值、方差归一化操作
x = torch.tensor([
    [ [1.0, 2.0, 3.0, 4.0],
      [5.0, 6.0, 7.0, 8.0] ],

    [ [2.0, 4.0, 6.0, 8.0],
      [1.0, 1.0, 1.0, 1.0] ]
])

layer_norm = nn.LayerNorm(normalized_shape=4)
out = layer_norm(x)
print(out)
# 作用：LayerNorm 的作用是：对每个 token 的向量做标准化，使其特征值更均衡、训练更稳定、更适合序列任务。
# 这里需要对比batch norm

tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]],

        [[-1.3416, -0.4472,  0.4472,  1.3416],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<NativeLayerNormBackward0>)


In [ ]:
# transformer 层之间堆叠的方式也要了解清楚，向量是怎么进行传递的？

## 生成式预训练预言模型


### 损失函数：GPT 训练目标：最大似然估计 vs 交叉熵损失（完整推导）

在 GPT 模型中，训练的核心目标是：给定一个文本序列，最大化该序列出现的概率。假设我们有一个长度为 $n$ 的 token 序列：

$$
w = (w_1, w_2, \dots, w_n)
$$

#### 🎯 一、最大似然估计（Maximum Likelihood Estimation, MLE）

我们希望学习模型参数 $\theta$，使得该序列在模型下的条件概率最大：

$$
P(w; \theta) = \prod_{i=1}^{n} P(w_i \mid w_{<i}; \theta)
$$

取对数后，得到对数似然（log-likelihood）：

$$
\log P(w; \theta) = \sum_{i=1}^{n} \log P(w_i \mid w_{<i}; \theta)
$$

**训练目标：最大化 log-likelihood，即最大似然估计（MLE）：**

$$
\mathcal{L}_{\text{MLE}}(\theta) = \max_{\theta} \sum_{i=1}^{n} \log P(w_i \mid w_{<i}; \theta)
$$

为了进行优化，我们将其转换为最小化负对数似然（Negative Log-Likelihood, NLL）：

$$
\mathcal{L}_{\text{NLL}}(\theta) = - \sum_{i=1}^{n} \log P(w_i \mid w_{<i}; \theta)
$$

---

#### 🔁 二、交叉熵损失（Cross-Entropy Loss）

在实现中，令 $P_{\text{true}}^{(i)}(v)$ 表示第 $i$ 个 token 的真实分布（通常是 one-hot 向量），
$P_{\text{pred}}^{(i)}(v)$ 表示模型在位置 $i$ 上的 softmax 概率输出。

对于第 $i$ 个 token，交叉熵为：

$$
\mathcal{L}_{\text{CE}}^{(i)} = - \sum_{v \in \mathcal{V}} P_{\text{true}}^{(i)}(v) \log P_{\text{pred}}^{(i)}(v)
$$

若真实标签是 one-hot，即 $P_{\text{true}}^{(i)}(v) = 1$ 当且仅当 $v = w_i$，则该式简化为：

$$
\mathcal{L}_{\text{CE}}^{(i)} = - \log P_{\text{pred}}^{(i)}(w_i)
$$

最终整条序列的交叉熵损失为：

$$
\mathcal{L}_{\text{CE}} = \sum_{i=1}^n \mathcal{L}_{\text{CE}}^{(i)} = - \sum_{i=1}^n \log P_{\text{pred}}^{(i)}(w_i)
$$

这正是负对数似然损失（NLL）的定义，因此：

$$
\mathcal{L}_{\text{CE}} = \mathcal{L}_{\text{NLL}}
$$


---

#### ✅ 三、结论：MLE 与交叉熵本质等价

| 理论名称                   | 数学表达                      | 实现名称                       | 是否等价 |
|----------------------------|-------------------------------|-------------------------------|----------|
| 最大似然估计（MLE）        | max log P(w_i \| w_{<i})      | -                             | ✅       |
| 负对数似然（NLL）          | - log P(w_i \| w_{<i})        | -                             | ✅       |
| 交叉熵损失（CrossEntropy） | - log P_pred(w_i)             | PyTorch中的 CrossEntropyLoss | ✅       |



因此，在 GPT 模型中，我们通常说：

> 模型训练目标是最大化对数似然，但实现中我们使用等价的交叉熵损失函数来最小化。

---

### 训练
在微调过程中，下游任务针对任务目标进行优化，很容易使得模型遗忘预训练阶段所学习的通
用语义知识表示，从而损失模型的通用性和泛化能力，导致出现灾难性遗忘（Catastrophic Forgetting）问题。因此，通常采用混合预训练任务损失和下游微调损失的方法来缓解上述问题